1. torch.nn命名空间提供了构建自己的神经网络所需的所有构建块。
2. PyTorch中的每个模块都是nn.module的子类。
3. 神经网络是由其他模块（层）组成的模块本身。这种嵌套结构允许轻松构建和管理复杂的体系结构。

在下面的部分中，我们将构建一个神经网络来对FashionMNIST数据集中的图像进行分类。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

使用torch.cuda is available判断是否可用GPU加速

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


我们通过对nn.Module进行子类化来定义我们的神经网络。模块，并在__init__中初始化神经网络层。

每个nn.Module子类在forward方法中实现对输入数据的操作。

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建了NeuralNetwork的一个实例，并将其移动到设备上，然后打印其结构。

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用模型，我们将输入数据传递给它。这将执行模型的forward以及一些后台操作。

在输入上调用模型会返回一个二维张量，dim=0对应于每个类的10个原始预测值的每个输出，dim=1对应于每个输出的单个值。我们通过将其传递给nn.Softmax模块的一个实例来获得预测概率。

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


让我们来分解FashionMNIST模型中的层。为了说明这一点，我们将使用3张尺寸为28x28的图片作为样本，看看在通过网络时会发生什么。

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


我们初始化nn.Flatten以将每个2D 28x28图像转换为784个像素值的连续阵列（保持小批量尺寸（dim=0））。

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


线性层nn.Linear是使用其存储的权重和偏差对输入应用线性变换的模块。

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


非线性激活是在模型的输入和输出之间创建复杂映射的原因。它们在线性变换后被应用于引入非线性，帮助神经网络学习各种现象。

使用ReLUJ激活函数

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}\n\n")

Before ReLU: tensor([[ 0.0466, -0.1624,  0.4153, -0.0371, -0.3682,  0.1618,  0.3354,  0.0048,
          0.2566, -0.6558,  0.2853, -0.2052, -0.1324,  0.3608,  0.2474, -0.0434,
          0.0686, -0.5062,  0.3249,  0.2683],
        [ 0.2245, -0.2510,  0.3030,  0.1618, -0.2358, -0.3639,  0.4615,  0.2633,
          0.2690, -0.4320,  0.2208, -0.3254, -0.0795,  0.1691,  0.2043,  0.0842,
          0.5477, -0.3101,  0.3475,  0.6510],
        [ 0.1259, -0.0109,  0.1554,  0.3019, -0.3158, -0.0515,  0.3052, -0.0199,
          0.1829, -0.5917,  0.1265, -0.1981,  0.1889,  0.1820,  0.2793,  0.0167,
          0.4231, -0.1300,  0.0905,  0.5969]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0466, 0.0000, 0.4153, 0.0000, 0.0000, 0.1618, 0.3354, 0.0048, 0.2566,
         0.0000, 0.2853, 0.0000, 0.0000, 0.3608, 0.2474, 0.0000, 0.0686, 0.0000,
         0.3249, 0.2683],
        [0.2245, 0.0000, 0.3030, 0.1618, 0.0000, 0.0000, 0.4615, 0.2633, 0.2690,
         0.0000, 0.2208, 0.0000, 0.0000, 0.1691, 0.204

nn.Sequential是有序的模块容器。数据按照定义的相同顺序传递给所有模块。

您可以使用顺序容器来组合一个快速网络，如seq_modules。

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

* 神经网络的最后一个线性层返回在[-infty，infty]范围中的logits-raw values
* 这些值传递nn.Softmax模块。逻辑被缩放为值[0，1]，表示每个类别的模型预测概率。
* dim参数指示值总和必须为1的维度。

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

神经网络中的许多层是参数化的，即具有在训练期间优化的相关权重和偏差。nn.Module子类自动跟踪模型对象内定义的所有字段，

你可以使用模型的parameters()、named_parameters()方法访问所有模型参数。

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : Parameter containing:
tensor([[-0.0309, -0.0324,  0.0021,  ..., -0.0270, -0.0280, -0.0287],
        [ 0.0345,  0.0199, -0.0039,  ..., -0.0196, -0.0219, -0.0317],
        [ 0.0233,  0.0138, -0.0333,  ..., -0.0098, -0.0091, -0.0162],
        ...,
        [ 0.0145, -0.0125,  0.0080,  ..., -0.0067, -0.0314,  0.0231],
        [-0.0310,  0.0182,  0.0298,  ..., -0.0282,  0.0204,  0.0044],
        [-0.0140,  0.0235,  0.0018,  ..., -0.0145, -0.0124,  0.0333]],
       device='cuda:0', requires_grad=True) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : Parameter conta